In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tutorial_utils import magma_to_verilog_string, smt_to_smtlib_string

import hwtypes as hw
import magma as m

<mark>In general, everything could use more elaboration.</mark>

The core of PEak is the python-embedded expression language of hwtypes. HWtypes defines abstract interfaces and type constructors for a number of types and kinds. This includes fixwidth bitvector, arbitrary precision floating point types, algebraic data types, and a bit (bool) type.  In this Section, we focus on the bit and bitvector type, the others are covered later.

A `BitVector` type of length `n` is constucted with `BitVector[n]`. `BitVector` has two key subtypes `Unsigned` and `Signed`. The author of this tutorial would have preferred for the mixing of signed and unsigned bitvector types to universally raise an error. However, in order to support legacy code, such mixing instead has undefined behavior which may raise errors.  Bitvector types have numerous constructor (also for legacy reasons). While the set of types accepted is not specifically defined by the abstract interface, bitvector implementations minimally support construction from: `BitVector`, `Bit`, `int`, types which define `__int__`, and sequences of objects that can be used to construct `Bit`. 

<mark> Caleb - The preceding is a little imprecise as what I mean is a bit/bitvector from the same implementation <mark> 

Similar to `BitVector` the full set of types that can be used to construct `Bit` types is implementation defined but minimally includes: `Bit`, `bool`, types which define `__bool__`, and the integer constants `0` and `1`. `Bit` types are required to implement the standard bitwise operators: and `&`, or `|`, xor `^`, and not `~`; equals `==` and not equals `!=`; and an if-then-else (`ite`) method which will describe later in this section.  

At a high level objects which have an `__int__` method can be thought of objects which are "castable" to `int`.  Readers should refer to the relevent python documentation for more details (https://docs.python.org/3/library/functions.html#int). Similarly the `__bool__` method is used define how objects are "cast" to `bool`, however, *all* objects are "castable" to `bool` unless they raise an error in `__bool__` or `__len__`.  See:  https://docs.python.org/3/library/functions.html#bool and https://docs.python.org/3/library/stdtypes.html#truth.
    
The SMT-LIB standard (http://smtlib.cs.uiowa.edu/index.shtml) defines a large set of arithmetic and bitwise functions on BitVectors.  These functions are defined in both the base theory (`FixedSizeBitVectors`) and in its associated logics (`BV` and `QF_BV`). The hwtypes bitvector interface defines a method for each of these functions with the exception of `bv2nat`.  For instance the moral equivalent of the smt term `(bvadd x y)` where `x` and `y` are terms of the sort `(_ BitVector n)` would be the hwtypes expression `x.bvadd(y)` where `x` and `y` are of the type `BitVector[n]`. More generally if `f` is a function defined by the SMT-LIB standard on bitvectors than there is equivalent method named `f` on the hwtypes `BitVector`. 
   
As a convenience these methods are defined as operators where applicable.  For example: `x.bvadd(y)` can be invoked with `x + y`. The semantics of sign dependent operators are defined by their type. For example `x < y` invokes `x.bvslt(y)` for signed `x` and `x.bvult(y)` for unsigned `y`.  Most operators and methods attempt to coerce their arguments. Any object that can be used to construct a bitvector will typically be coerced. This was done to allow the use of python integer constants e.g. `bv + 1`.

Hwtypes provides two main implementations of the bitvector and bit types.  The first implementation is a pure python functional model over constant values. The second wraps pySMT (described bellow) to generate SMT terms. Finaly, Magma provides a third implementation which allows for the definition of circuits. This uniform interface allows for the same hwtypes programs to be interpretted multiple ways.  First, the pure python implementation can be use to simulate a circuit, the SMT implementation can be used to generate a formal model, and the last to generate actual rtl.  This single source of truth is powerful for designers as they need not implement the same thing multiple times. We believe such a single source of truth should act as executable specification. When specifications are written in a non executable form there is a lot of room for engineer interpretation.  When it comes time to test a design and a functional model formal model, or RTL differ we are left wondering where the error is?  Did all teams interpret specification correctly? If not whose implimentation is wrong?  With an executable spec such questions become trivial.     

pySMT is a solver indepented API for constructing SMT formulae. The use of pySMT allows a user to use their SMT solver of choice. Importantly, pySMT allows constructing SMT formulae without a solver.  This allows the constructors of the different BitVector implementations to be uniform. In contrast, SMT-Switch (a similar project) requires a solver object to to build terms. This would be quite inconvient for hwtypes as either a reference to a solver would need to be passed to each bitvector (changing the constructor interface) or there would be need to be an implicit global context which holds a reference to a solver (which would make working with multiple solvers difficult).


<mark>Say more about single source of truth.  Why is it important?  What's an example where not having it is a problem?</mark>

<mark> Don't totally understand this. What is "pure python"?  What would the alternative be?</mark>
    
<mark> Caleb - the alternative is AST rewriting.  A sufficiently sophisticated python developer would realize something very strange is going in PEak. </mark>    
 
Hwtypes is an expression language only, all statements are executed in pure python following typical python semantics. In Section 2 we will introduce the Language PEak which break away from semantics of pure python and reinterprets the meaning of assignment statements and if statements.
    
    

In the following we see how the same function can be invoked with constant python values, symbolic smt values, or to build a magma circuit:

In [ ]:
def add(x, y):
    return x + y

In [ ]:
PyDataT = hw.BitVector[8]
SmtDataT = hw.SMTBitVector[8]
MagmaDataT = m.Bits[8]

x = PyDataT(1)
y = PyDataT(2)
results = add(x, y)
print(repr(results))
print('---')

x = SmtDataT(name='x')
y = SmtDataT(name='y')
results = add(x, y)
print(smt_to_smtlib_string(results))
print('---')
# del because jupyter seems to keep references alive longer than it should which breaks SMT variables
del x
del y


class Adder(m.Circuit):
    io = m.IO(
        x=m.In(MagmaDataT), y=m.In(MagmaDataT), results=m.Out(MagmaDataT)
    )
    io.results @= add(io.x, io.y)


print(magma_to_verilog_string(Adder))
print('---')

The real power of hwtypes comes from its embedding in python which facilitates the generation of more complex formulas. For example, we can generalize add to build an adder tree over any number of inputs with the use of a recursive function:

In [ ]:
def add_n(*args):
    n = len(args)
    if n == 0:
        return 0
    elif n == 1:
        return args[0]
    else:
        return add_n(*args[:n // 2]) + add_n(*args[n // 2:])

In [ ]:
x = PyDataT(1)
y = PyDataT(2)
z = PyDataT(3)
results = add_n(x, y, z)
print(repr(results))
print('---')

x = SmtDataT(name='x')
y = SmtDataT(name='y')
z = SmtDataT(name='z')
results = add_n(x, y, z)
print(smt_to_smtlib_string(results))
print('---')
del x
del y
del z


class Adder3(m.Circuit):
    io = m.IO(
        x=m.In(MagmaDataT),
        y=m.In(MagmaDataT),
        z=m.In(MagmaDataT),
        results=m.Out(MagmaDataT)
    )
    io.results @= add_n(io.x, io.y, io.z)


print(magma_to_verilog_string(Adder3))
print('---')

We can easily further generalize this to build reduction trees over any function. 

In [ ]:
_MISSING = object()  # a sentinel object


def gen_tree_reducer(f, ident=_MISSING):
    '''
    f :: T -> T -> T
    ident :: Optional[T]
    '''
    def reducer(*args):
        '''
         *args :: List[T]
        '''
        n = len(args)
        if n == 0:
            if ident is _MISSING:
                raise ValueError('cannot reduce empty list')
            return ident
        elif n == 1:
            return args[0]
        else:
            return f(reducer(*args[:n // 2]), reducer(*args[n // 2:]))

    return reducer

Using builtin higher order functions (e.g. `map`) is also supported

In [ ]:
add_n = gen_tree_reducer(add, 0)


def sum_of_sq(*args):
    return add_n(*map(lambda x: x * x, args))

In [ ]:
x = PyDataT(1)
y = PyDataT(2)
z = PyDataT(3)
results = sum_of_sq(x, y, z)
print(repr(results))
print('---')

x = SmtDataT(name='x')
y = SmtDataT(name='y')
z = SmtDataT(name='z')
results = sum_of_sq(x, y, z)
print(smt_to_smtlib_string(results))
print('---')
del x
del y
del z


class SumOfSq3(m.Circuit):
    io = m.IO(
        x=m.In(MagmaDataT),
        y=m.In(MagmaDataT),
        z=m.In(MagmaDataT),
        results=m.Out(MagmaDataT)
    )
    io.results @= sum_of_sq(io.x, io.y, io.z)


print(magma_to_verilog_string(SumOfSq3))
print('---')

Note how all `if`'s in `gen_tree_reducer` are resolved without respect to the value's of the data. The hwtypes expression language allows for conditionals using the `ite` method on the bit type.  For example one might write an absolute value function as follows:  

In [ ]:
def abs(x):
    return (x < 0).ite(-x, x)

In [ ]:
PyDataT = hw.SIntVector[8]
SmtDataT = hw.SMTSIntVector[8]
MagmaDataT = m.SInt[8]

x = PyDataT(-1)
results = abs(x)
print(repr(results))
print('---')

x = SmtDataT(name='x')
results = abs(x)
print(smt_to_smtlib_string(results))
print('---')
del x


class Abs(m.Circuit):
    io = m.IO(x=m.In(MagmaDataT), results=m.Out(MagmaDataT))
    io.results @= abs(io.x)


print(magma_to_verilog_string(Abs))
print('---')

It is important to note that only constantly bounded recursion is possible. This ensures all hwtypes programs may be compiled to some finite circuit (and correspondingly some finite formula in first order logic). This restriction is easy to enforce as the `ite` method behaves like any other python method call. In particular its arguments will be evaluated eagerly, meaning unbounded data dependent recursion will recurse infinitely:

In [ ]:
def factorial(x):
    return (x == 1).ite(
        x * factorial(
            x - 1
        ),  # factorial(x - 1) will always be evaluated leading to infinite recursion
        type(x)(1),  # cast 1 to the type of x
    )


try:
    x = factorial(PyDataT(5))
except RecursionError as e:
    print(f'Error: {e}')
else:
    print(f'5! = {x}')

Now the above program could be unrolled explicitly, which results in a quite large but finite circuit.  

In [ ]:
PyDataT = hw.UIntVector[8]

def bounded_factorial(x):
    if not isinstance(x, hw.AbstractBitVector):
        raise TypeError()
    T = type(x)
    MAX_INT = 2**x.size - 1

    def inner(x, ctr):
        if ctr == 0:
            return T(1)
        else:
            return (x <= 1).ite(
                T(1),
                x * inner(x - 1, ctr - 1),
            )

    return inner(x, MAX_INT)


x = PyDataT(9)
try:
    y = bounded_factorial(x)
except RecursionError as e:
    print(f'Error: {e}')
else:
    print(f'{x}! = {y}')

Note this circuit in fact does not perform the factorial function. Instead it performs: 
$$
f(0) = 1 \\
f(x) = x*f(x-1) \mod 2^{\text{bitwidth}(x)}$$